In [ ]:
from elasticsearch import Elasticsearch
import json 

In [ ]:
# Create elasticsearch connection
client = Elasticsearch("https://localhost:9200/", basic_auth=("elastic", "txx7ce39UVrCvqcwL77f"), verify_certs=False)
client.info()

In [ ]:
# Load data tinh_tp
with open('./data_address/tinh_tp.json', 'r', encoding="utf") as file:
    data = json.load(file)

for i in data:
    data[i]["id"] = i
    data[i]["abbreviation"] = ' '.join([''.join([c.lower()[0] for c in data[i]["name_with_type"].split(' ')]), ''.join([c.lower()[0] for c in data[i]["name"].split(' ')])])
    resp = client.index(index="tinh_tp", id=i, document=data[i])
    print(resp["result"])
    


In [ ]:
# Load data quan_huyen
with open('./data_address/quan_huyen.json', 'r', encoding="utf") as file:
    data = json.load(file)

for i in data:
    data[i]["id"] = i
    data[i]["abbreviation"] = ' '.join([''.join([c.lower()[0] if len(c) > 0 else '' for c in data[i]["name_with_type"].split(' ')]), ''.join([c.lower()[0] if len(c) > 0 else '' for c in data[i]["name"].split(' ')])])
    resp = client.index(index="quan_huyen", id=i, document=data[i])
    print(resp["result"])

In [ ]:
# Load data xa_phuong
with open('./data_address/xa_phuong.json', 'r', encoding="utf") as file:
    data = json.load(file)

for i in data:
    data[i]["id"] = i
    data[i]["abbreviation"] = ' '.join([''.join([c.lower()[0] if len(c) > 0 else '' for c in data[i]["name_with_type"].split(' ')]), ''.join([c.lower()[0] if len(c) > 0 else '' for c in data[i]["name"].split(' ')])])
    resp = client.index(index="xa_phuong", id=i, document=data[i])
    print(resp["result"])

In [ ]:
# Test search
resp = client.search(
    index="tinh_tp",
    query={
        "multi_match": {
            "query": "tphcm",
            "fields": ["parent_code","name","slug","name_with_type","path_with_type","abbreviation"]
        }
    },
)
resp["hits"]["hits"]
# if len(resp["hits"]["hits"]) > 0:
#     print()
# else:
#     print("Not found")

In [ ]:
import pandas as pd
df = pd.DataFrame(pd.read_csv("./output.csv"))
df_no_duplicates = df.drop_duplicates(subset=df.columns.difference(['id']))
df_no_duplicates.drop('title', axis=1, inplace=True)
df_no_duplicates

In [ ]:

df_filtered = df_no_duplicates[df_no_duplicates['province'].isin([1, 79, 31, 92, 48])]
df_filtered = df_filtered[df_filtered['is_rent'] == 0]
df_filtered.to_csv('./output_cleaned.csv', index=False)